In [63]:
from azureml.core import Workspace, Dataset, Experiment, Run
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute_target import ComputeTargetException
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import logging, requests,json
import urllib.request

# get/create experiment
ws = Workspace.from_config()

# get dataset
dataset = Dataset.get_by_name(ws, name='wine')
df = dataset.to_pandas_dataframe()

In [53]:
#split dataset
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
label = "quality"

#Saves the train and test dataset in a Tabular type
datastore = ws.get_default_datastore()
train_dataset = Dataset.Tabular.register_pandas_dataframe(train_data,datastore,'wine_train_dataset',description='Train dataset for wine')
test_dataset = Dataset.Tabular.register_pandas_dataframe(test_data,datastore,'wine_test_dataset',description='Test dataset for wine')


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/8f8d2a94-38ba-42f2-bf38-e5058246ba7c/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/b17f079f-95c6-440d-8ea3-f974bb2a23eb/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [54]:
#Compute

compute_name = 'gusciancnotebook'

try:
    compute_target = ComputeTarget(workspace=ws,name=compute_name)
    print('Found existing compute resource, using it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size= 'STANDARD_D12_V2',
        max_nodes=2
    )
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing compute resource, using it

Running


In [55]:
#Automl config
automl_config = AutoMLConfig(
                                 task='classification',
                                 debug_log ='automl_notebook_errors.log',
                                 primary_metric='AUC_weighted',
                                 experiment_timeout_minutes=30,
                                 training_data=train_dataset,
                                 label_column_name=label,
                                 compute_target = compute_target,
                                 n_cross_validations=5,
                                 verbosity=logging.INFO,
                                 featurization='auto',
                                 enable_voting_ensemble=False,
                                 enable_stack_ensemble=False
                                )

In [56]:
#Experiment set up
experiment_name = 'AutoML-Notebook'
experiment = Experiment(ws, experiment_name)

#Execute run
run = experiment.submit(automl_config, show_output=True)

best_run, fitted_model = run.get_output()
print(fitted_model.steps)

Submitting remote run.
No run_configuration provided, running on gusciancnotebook with default configuration
Running on remote compute: gusciancnotebook


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-Notebook,AutoML_5e5472eb-8710-4289-9ecb-2058212fd094,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

In [80]:
#Test model classification
y_test = test_data.pop("quality")

y_classify = fitted_model.predict(test_data)

print("Predicted:  " + y_classify[:10] + " Value: " + y_test[:10])


803      Predicted:  bad Value: good
124       Predicted:  bad Value: bad
350      Predicted:  bad Value: good
682       Predicted:  bad Value: bad
1326    Predicted:  good Value: good
976       Predicted:  bad Value: bad
1493      Predicted:  bad Value: bad
706       Predicted:  bad Value: bad
613      Predicted:  good Value: bad
1587    Predicted:  good Value: good
Name: quality, dtype: object


In [57]:
#Register model
model_name = best_run.properties['model_name']
description = 'AutoML jupyter example'
tags = None

model = run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

In [59]:

script_file_name = 'score.py'
file_dwn = best_run.download_file('./outputs/scoring_file_v_1_0_0.py', script_file_name)

print(file_dwn)

None


In [60]:
#Deploy model into container
inference_config = InferenceConfig(environment = best_run.get_environment(), 
                                   entry_script = './score.py')

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 4, 
                                               tags = {'type': "automl-jupyter"},
                                               description = "Automl jupyter sample service")

aci_service_name = 'automl-jupyter-model'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-jupyter-model
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-11 15:07:32+00:00 Creating Container Registry if not exists.
2021-10-11 15:07:32+00:00 Registering the environment.
2021-10-11 15:07:32+00:00 Use the existing image.
2021-10-11 15:07:33+00:00 Generating deployment configuration.
2021-10-11 15:07:34+00:00 Submitting deployment to compute..
2021-10-11 15:07:47+00:00 Checking the status of deployment automl-jupyter-model..
2021-10-11 15:11:48+00:00 Checking the status of inference endpoint automl-jupyter-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [83]:
scoring_uri = aci_service.scoring_uri
scoring_uri

'http://4e6aa4d1-bd2d-4bb3-b9d8-d22d3655ba9c.westeurope.azurecontainer.io/score'

In [115]:
# Request data goes here
data = {"data": [
    {
      "fixed acidity": 8.8,
      "volatile acidity": 0.32,
      "citric acid": 0.57,
      "residual sugar": 2.3,
      "chlorides": 0.069,
      "free sulfur dioxide": 29,
      "total sulfur dioxide": 37,
      "density": 0.997,
      "pH": 3.15,
      "sulphates": 0.61,
      "alcohol": 9.6
    }
  ],
  "method": "predict"
  }

input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print(resp.status_code)
print("The wine submitted is " + resp.json())

200
The wine submitted is {"result": ["bad"]}
